<center>**Equation to calculate weighted allele frequency. Only shown for 'A' allele. Same equation for 'B' allele. (See 'allele_next_gen' function)**
<br><br>
\begin{equation*}
\left(pA_{weighted}  \right) = \frac{\left(\sum_{k=1}^n pA_k \times N_k \right)} {\left( \sum_{k=1}^n N_k \right)}
\end{equation*}

<center>**Equation to calculate weighted migration rate. (See 'allele_next_gen' function)** 
<br><br>
\begin{equation*}
\left(m_{weighted}  \right) = \frac{\left(\sum_{j=1}^n m_{ij} \times N_j \right)} {\left( \sum_{j=1}^n N_j \right)}
\end{equation*}

In [170]:
import numpy as np

In [171]:
# Function used to determine migration rates based on distances between all populations. 
# Linear decrease in migration rate with increasing distance. Rate of decreased based on slope ('m')
# which depends on the maximum distance between populations and the desired 'max_mig_rate' (i.e. migration rate when distace = 0)
def migration_rate(Distance_Dic, max_mig_rate):
    '''
    ## PARAMETERS ##
    
    Distance_Dic: Dictionary containing distances between all pairwise combinations of cells in 'Matrix'. 
    Will also contain migration rates after use of this function
    
    max_mig_rate: Desired migration rate when distance = 0
        
    ## USED IN FUNCTIONS ##
    
    1. Distance_Mig
    '''  
    max_dis = max(Distance_Dic.values()) # Max distance between populations
    m = (max_mig_rate - 0)/(max_dis[0] - 0) # Slope. Assumes close to no migration at max distance. Relized migration at max distance may be slightly greater than 0 due to rounding. 
    for Dkey, Dvalue in Distance_Dic.items():
        Mig_prop = max_mig_rate - m*Dvalue[0] 
        Distance_Dic[Dkey].append(round(Mig_prop,3)) # Append migration rate to 'Distance_Dic' dictionary
        
# Function to calculate distances between all pairwise combinations of cells in matrix. Also calls
# the migration rate function and appends migration rate this to the distance dictionary. 
def Distance_Mig(max_mig_rate):
    '''
    ## PARAMETERS ##
    
    Matrix: m x n dimensional matrix, initialized at the outset of each simulation. 
    Contains empty cells that may become filled with populations. 
    
    max_mig_rate: Desired migration rate when distance = 0
        
    ## USED IN FUNCTIONS ##
    
    1. Simulate
    ''' 
    Matrix = np.zeros((5, 5), dtype = 'int')
    rows = [i for i in range(len(Matrix))] 
    cols = [i for i in range(len(Matrix))]
    matelem = [(i,j) for i in rows for j in cols]
    dis = [[i,j] for i in matelem for j in matelem]
    for i in dis:
        if i[0][0] == i[1][0]: # Within rows
            dis1 = abs(i[1][1] - i[0][1])
            i.append(dis1)
        elif i[0][1] == i[1][1]: # Within columns
            dis2 = abs(i[1][0] - i[0][0])
            i.append(dis2)    
        else: # Diagonals
            dis3 = (((i[1][1] - i[0][1])**2) + ((i[1][0] - i[0][0])**2))**(0.5)
            i.append(dis3)
    global Distance_Dic
    Distance_Dic = {'{0}.{1}'.format(key1,key2):[round(key3,2)] for key1,key2,key3 in dis}
    migration_rate(Distance_Dic, max_mig_rate)

# Function used to determine the frequency of 'A' and 'B' alleles in the infinite allele pool to be sampled the following
# generation. Uses the formula for calculating the frequency of alleles in the next generation under migration (i.e. pA1 = pA0*[1 - m] + pAm*m)
# The frequency of alleles in the metpopulation (i.e. pAm) is the mean allele frequency across all populations, weighted by their population size
# The migration rate is sismilarly the mean migration rate across all populations, weighted by population size.
# I did not weight by distance since this is implicit in the migration rates. 
def alleles_next_gen(Akey, pop_list, alleles, Matrix):
    '''
    ## PARAMETERS ##
        
    Akey: Index used to cycle through keys in alleles dictionary. 
    Note keys correspond to populations. See 'cline' function.  
    
    pop_list: list containing all current populations in existence
    
    alleles: Dictionary used to stored lists of alleles population size for each population. 
    Updated every generation.
    
    Matrix: m x n dimensional matrix, initialized at the outset of each simulation. 
    Contains empty cells that may become filled with populations.
        
    ## USED IN FUNCTIONS ##
    
    1. cline
    ''' 
    to_pop = (np.where(Matrix == int(Akey))[0][0], np.where(Matrix == int(Akey))[1][0]) # Location of focal population in matrix. 
    migration_weighted = [] # List holding migration rates
    allele_weighted_A = [] # List holding frequency of 'A' alleles
    allele_weighted_B = [] # List holding frequency of 'B' alleles
    Size = [] # List holding population sizes
    for i in pop_list: 
        if Akey == i: 
            pass
        else:
            from_pop = (np.where(Matrix == int(i))[0][0], np.where(Matrix == int(i))[1][0]) # Location of source population in matrix
            con = str(to_pop) + '.' + str(from_pop) # Create concatenated string from current focal population and source population                
            migration_weighted.append(Distance_Dic[con][1]) # Append migration rate to list
            allele_weighted_A.append(allele_freq(alleles[i]['A'])) # Append allele 'A' frequency to list
            allele_weighted_B.append(allele_freq(alleles[i]['B'])) # Append allele 'B' frequency to list
            Size.append(alleles[i]['S'][0]) # Append population size to list
    migration_weighted = sum(migration_weighted[g] * Size[g] / sum(Size) for g in range(len(migration_weighted))) # Weighted migration rate
    allele_weighted_A = sum(allele_weighted_A[g] * Size[g] / sum(Size) for g in range(len(allele_weighted_A))) # Weighted allele 'A'
    allele_weighted_B = sum(allele_weighted_B[g] * Size[g] / sum(Size) for g in range(len(allele_weighted_B))) # Weighted allele 'B'
    pA1 = ((1 - migration_weighted) * allele_freq(alleles[Akey]['A'])) + (migration_weighted * allele_weighted_A) # Probability of sampling 'A' in next generation
    pB1 = ((1 - migration_weighted) * allele_freq(alleles[Akey]['B'])) + (migration_weighted * allele_weighted_B) # Probability of sampling 'B' in next generation
    return migration_weighted, allele_weighted_A, allele_weighted_B, pA1, pB1

# From list containing alleles, calculate the frequency of 'A' or 'B' allele. 
def allele_freq(locus):
    '''
    ## PARAMETERS ##
    
    locus: List containing alleles
    
    ## USED IN FUNCTIONS ##
    
    1. cline
    '''
    p = sum(1*i.isupper() for i in locus)/float(len(locus))
    return p

In [172]:
# Let's first define our matrix. For illustration this will be a small, 5 x 5 matrix
# This is only for illustration. The 'Distance_Mig' function actually creates the matrix as a local variable. 
Matrix = np.zeros((5, 5), dtype = 'int')

Matrix

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])

In [173]:
# Next let's call our 'Distance_Mig' function to generate the dictionary that stores pairwise distances and migration rates. 
Distance_Mig(max_mig_rate = 0.5)

# Global dictionary output from above function. For each pairwise combination of populations : [Distance, migration rate]
# Thus, closer populations exchange a greater proportion of alleles. 
Distance_Dic 

{'(0, 0).(0, 0)': [0.0, 0.5],
 '(0, 0).(0, 1)': [1.0, 0.412],
 '(0, 0).(0, 2)': [2.0, 0.323],
 '(0, 0).(0, 3)': [3.0, 0.235],
 '(0, 0).(0, 4)': [4.0, 0.147],
 '(0, 0).(1, 0)': [1.0, 0.412],
 '(0, 0).(1, 1)': [1.41, 0.375],
 '(0, 0).(1, 2)': [2.24, 0.302],
 '(0, 0).(1, 3)': [3.16, 0.221],
 '(0, 0).(1, 4)': [4.12, 0.136],
 '(0, 0).(2, 0)': [2.0, 0.323],
 '(0, 0).(2, 1)': [2.24, 0.302],
 '(0, 0).(2, 2)': [2.83, 0.25],
 '(0, 0).(2, 3)': [3.61, 0.181],
 '(0, 0).(2, 4)': [4.47, 0.105],
 '(0, 0).(3, 0)': [3.0, 0.235],
 '(0, 0).(3, 1)': [3.16, 0.221],
 '(0, 0).(3, 2)': [3.61, 0.181],
 '(0, 0).(3, 3)': [4.24, 0.125],
 '(0, 0).(3, 4)': [5.0, 0.058],
 '(0, 0).(4, 0)': [4.0, 0.147],
 '(0, 0).(4, 1)': [4.12, 0.136],
 '(0, 0).(4, 2)': [4.47, 0.105],
 '(0, 0).(4, 3)': [5.0, 0.058],
 '(0, 0).(4, 4)': [5.66, 0.0],
 '(0, 1).(0, 0)': [1.0, 0.412],
 '(0, 1).(0, 1)': [0.0, 0.5],
 '(0, 1).(0, 2)': [1.0, 0.412],
 '(0, 1).(0, 3)': [2.0, 0.323],
 '(0, 1).(0, 4)': [3.0, 0.235],
 '(0, 1).(1, 0)': [1.41, 0.375],


In [174]:
# Let's create a dictionary with some populations. This dictionary is identical to that used in the simulations.
# The only difference is the simulations start with a single population and other are created whereas here I am
# initializing all populations for the purpose of illustrating how migration is happening. 

# 'A' = "A" locus list, 'B' = "B" locus list, 'S' = Population size
alleles = {'1': {'A' : (['A'] * int(10 * 0.5) ) + (['a'] * int(round(10 * 0.5))), 
                 'B' : (['B'] * int(10 * 0.5) ) + (['b'] * int(round(10 * 0.5))), 
                 'S' : [10]},
           '2' : {'A' : (['A'] * int(10 * 1.0) ) + (['a'] * int(round(10 * 0))), 
                  'B' : (['B'] * int(10 * 0) ) + (['b'] * int(round(10 * 1.0))), 
                  'S' : [10]},
           '3' : {'A' : (['A'] * int(10 * 0.2) ) + (['a'] * int(round(10 * 0.8))), 
                  'B' : (['B'] * int(10 * 0.2) ) + (['b'] * int(round(10 * 0.8))), 
                  'S' : [10]}}

alleles

{'1': {'A': ['A', 'A', 'A', 'A', 'A', 'a', 'a', 'a', 'a', 'a'],
  'B': ['B', 'B', 'B', 'B', 'B', 'b', 'b', 'b', 'b', 'b'],
  'S': [10]},
 '2': {'A': ['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A'],
  'B': ['b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b'],
  'S': [10]},
 '3': {'A': ['A', 'A', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a'],
  'B': ['B', 'B', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b'],
  'S': [10]}}

In [175]:
# Let's now manually add these populations to the matrix. Again, this is normally done automatically upon creation
# of a new population

Matrix[0, 0] = 1
Matrix[0, 1] = 2
Matrix[1, 2] = 3

Matrix

array([[1, 2, 0, 0, 0],
       [0, 0, 3, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])

In [176]:
# A simple function that will itereate over the alleles lists (i.e. population allele pools) and run the 'alleles_next_gen'
# This will return one tuple for each population as (frequency of 'A' next gen, frequency of 'B' next gen).
def loop():
    for Akey, Avalue in alleles.items():
        print Akey, alleles_next_gen(Akey, pop_list, alleles, Matrix)

In [177]:
pop_list = alleles.keys()
loop()

# Output: (weighted migration rate, weighted frequency of 'A', weighted frequency of 'B', next gen allele 'A' frequency, next gen allele 'B' frequency)

1 (0.357, 0.6, 0.1, 0.5357, 0.3572)
3 (0.3385, 0.75, 0.25, 0.38617500000000005, 0.216925)
2 (0.3935, 0.35, 0.35, 0.744225, 0.137725)


In [169]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
